# HIVE: converting ABF to HDF5

In [1]:
from hive.convert.abf2h5 import ABFConverter

abf = ABFConverter(
    input_file='/data/hnl/in-vitro/datasets/2019_05_13_DA_uncorrelated_100k_97Hz_MMP001ktk1a+MMP001ktk1b/2019_05_13_DA_uncorrelated_100k_97Hz_MMP001ktk1a+MMP001ktk1b_0000.abf', 
    # output_file='', 
    channel_select=[],
    verbose=True)

abf.process()

read 2019_05_13_DA_uncorrelated_100k_97Hz_MMP001ktk1a+MMP001ktk1b_0000.abf: 0.557641 seconds
	wrote header: 0.015608 seconds
	wrote FSCV_1: 1.103538 seconds
	wrote Cmd_1: 0.844563 seconds
	wrote FSCV_2: 1.055668 seconds
	wrote Cmd_2: 0.930613 seconds
wrote 2019_05_13_DA_uncorrelated_100k_97Hz_MMP001ktk1a+MMP001ktk1b_0000.h5: 3.969667 seconds
